## Flights API

In [1]:
import requests
import json
import re
from pyowm.owm import OWM
import pandas as pd
import datetime
from IPython.display import JSON

In [ ]:
icao = 'EDDB' #berlin germany
tomorrow = (datetime.datetime.today() + datetime.timedelta(days=1)).isoformat()[:10]
tomorrow

'2022-04-18'

### Connect to flights Api

In [3]:
# code copied from API:
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T00:00/{tomorrow}T12:00"
querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true",
               "withPrivate":"false","withLocation":"false"}
headers = {
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
    "X-RapidAPI-Key": '542b426faamsh04f22b0d9747b0cp11f2c0jsn8e3a0be981a2'
}
response = requests.request("GET", url, headers=headers, params=querystring)
    


In [4]:
response.json()

{'departures': [{'departure': {'scheduledTimeLocal': '2022-04-18 06:00+02:00',
    'scheduledTimeUtc': '2022-04-18 04:00Z',
    'terminal': '0',
    'quality': ['Basic']},
   'arrival': {'airport': {'icao': 'LGRP',
     'iata': 'RHO',
     'name': 'Rodes Island'},
    'scheduledTimeLocal': '2022-04-18 10:15+03:00',
    'scheduledTimeUtc': '2022-04-18 07:15Z',
    'quality': ['Basic']},
   'number': 'FR 351',
   'status': 'Unknown',
   'codeshareStatus': 'Unknown',
   'isCargo': False,
   'aircraft': {'model': 'Boeing 737-800'},
   'airline': {'name': 'Ryanair'}},
  {'departure': {'scheduledTimeLocal': '2022-04-18 06:00+02:00',
    'scheduledTimeUtc': '2022-04-18 04:00Z',
    'terminal': '1',
    'quality': ['Basic']},
   'arrival': {'airport': {'icao': 'LPPT', 'iata': 'LIS', 'name': 'Lisbon'},
    'scheduledTimeLocal': '2022-04-18 08:50+01:00',
    'actualTimeLocal': '2022-04-18 08:50+01:00',
    'scheduledTimeUtc': '2022-04-18 07:50Z',
    'actualTimeUtc': '2022-04-18 07:50Z',
    'te

### Unpack this horrendous list of lists of dictionaries

In [5]:
# Step 1: make a dataframe from all of the information in the json
arrivals = pd.json_normalize(response.json()['arrivals'])

# Step2: select only the columns you want to incude on your database
json_norm_df = pd.DataFrame(arrivals
    .filter(['departure.airport.name','arrival.scheduledTimeLocal',
             'arrival.terminal','status','aircraft.model','number'])
    .assign(icao_code = icao))

json_norm_df

,departure.airport.name,arrival.scheduledTimeLocal,arrival.terminal,status,aircraft.model,number,icao_code
0,Duesseldorf,2022-04-18 07:35+02:00,1,Unknown,Airbus A319,EW 9048,EDDB
1,Munich,2022-04-18 07:35+02:00,1,Unknown,Airbus A319,LH 1926,EDDB
2,Salzburg,2022-04-18 07:40+02:00,1,Unknown,Airbus A319,EW 4342,EDDB
3,Budapest,2022-04-18 07:45+02:00,0,Unknown,Airbus A320,W6 2315,EDDB
4,Newark,2022-04-18 07:50+02:00,1,Unknown,Boeing 767-300,UA 962,EDDB
...,...,...,...,...,...,...,...
62,London,2022-04-18 11:40+02:00,1,Unknown,Airbus A319,BA 982,EDDB
63,Reykjavik,2022-04-18 11:40+02:00,1,Unknown,Airbus A320 NEO,OG 700,EDDB
64,Stockholm,2022-04-18 11:45+02:00,1,Unknown,Airbus A320,U2 5882,EDDB
65,London,2022-04-18 11:45+02:00,1,Unknown,Airbus A319,U2 8209,EDDB


In [6]:
json_norm_df['arrival.scheduledTimeLocal'] = pd.to_datetime(json_norm_df['arrival.scheduledTimeLocal'])
json_norm_df.dtypes

departure.airport.name                                       object
arrival.scheduledTimeLocal    datetime64[ns, pytz.FixedOffset(120)]
arrival.terminal                                             object
status                                                       object
aircraft.model                                               object
number                                                       object
icao_code                                                    object
dtype: object

In [35]:
json_norm_df.rename(columns={'departure.airport.name':'departing_from',
                             'arrival.scheduledTimeLocal':'arrival_time_local',
                             'arrival.terminal':'terminal',
                             'aircraft.model':'aircraft_model',
                            'number':'flight_number'}, inplace=True)

json_norm_df['arrival_time_local'] = pd.to_datetime(json_norm_df['arrival_time_local'])
arrivals = json_norm_df
arrivals.head()

,departing_from,arrival_time_local,terminal,status,aircraft_model,flight_number,icao_code
0,Duesseldorf,2022-04-18 07:35:00+02:00,1,Unknown,Airbus A319,EW 9048,EDDB
1,Munich,2022-04-18 07:35:00+02:00,1,Unknown,Airbus A319,LH 1926,EDDB
2,Salzburg,2022-04-18 07:40:00+02:00,1,Unknown,Airbus A319,EW 4342,EDDB
3,Budapest,2022-04-18 07:45:00+02:00,0,Unknown,Airbus A320,W6 2315,EDDB
4,Newark,2022-04-18 07:50:00+02:00,1,Unknown,Boeing 767-300,UA 962,EDDB


### Connecting to OWM Weather API

In [9]:
import requests

OWM_key = 'b556bcb3a0753fe0399a44800d14a3c2'
city = "Berlin"
country = "DE"
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')
response

<Response [200]>

In [10]:
forecast_api = response.json()['list']

weather_info = []

# datetime, temperature, wind, prob_pers, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city + ',' + country
    weather_info.append(weather_hour)
    
weather_data = pd.DataFrame(weather_info)
weather_data.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-04-17 15:00:00,15.34,3.19,0.0,0.0,0,"Berlin,DE"
1,2022-04-17 18:00:00,13.81,3.34,0.0,0.0,0,"Berlin,DE"
2,2022-04-17 21:00:00,9.89,3.18,0.0,0.0,0,"Berlin,DE"
3,2022-04-18 00:00:00,5.52,2.31,0.0,0.0,0,"Berlin,DE"
4,2022-04-18 03:00:00,4.34,2.07,0.0,0.0,0,"Berlin,DE"


In [11]:
weather_info

[{'datetime': '2022-04-17 15:00:00',
  'temperature': 15.34,
  'wind': 3.19,
  'prob_perc': 0.0,
  'rain_qty': 0,
  'snow': 0,
  'municipality_iso_country': 'Berlin,DE'},
 {'datetime': '2022-04-17 18:00:00',
  'temperature': 13.81,
  'wind': 3.34,
  'prob_perc': 0.0,
  'rain_qty': 0,
  'snow': 0,
  'municipality_iso_country': 'Berlin,DE'},
 {'datetime': '2022-04-17 21:00:00',
  'temperature': 9.89,
  'wind': 3.18,
  'prob_perc': 0.0,
  'rain_qty': 0,
  'snow': 0,
  'municipality_iso_country': 'Berlin,DE'},
 {'datetime': '2022-04-18 00:00:00',
  'temperature': 5.52,
  'wind': 2.31,
  'prob_perc': 0.0,
  'rain_qty': 0,
  'snow': 0,
  'municipality_iso_country': 'Berlin,DE'},
 {'datetime': '2022-04-18 03:00:00',
  'temperature': 4.34,
  'wind': 2.07,
  'prob_perc': 0.0,
  'rain_qty': 0,
  'snow': 0,
  'municipality_iso_country': 'Berlin,DE'},
 {'datetime': '2022-04-18 06:00:00',
  'temperature': 5.96,
  'wind': 2.82,
  'prob_perc': 0.0,
  'rain_qty': 0,
  'snow': 0,
  'municipality_iso_co

## Population data

## Airports data

In [12]:
import pandas as pd

airports_cities = (
pd.read_csv('airports.csv')
    .query('type == "large_airport"')
    .filter(['name','iso_country','municipality','gps_code'])
    .rename(columns={'gps_code':'icao_code'})
    .assign(municipality_iso_country = lambda x: x['municipality'] + ',' + x['iso_country'])
)
airports_cities.head(1)

,name,iso_country,municipality,icao_code,municipality_iso_country
10890,Honiara International Airport,SB,Honiara,AGGH,"Honiara,SB"


In [ ]:
(
airports_cities
    .merge(arrivals, on='icao_code', how='inner')
    .merge(weather_data, on='municipality_iso_country', how='inner')
    .head()
)

,name,iso_country,municipality,icao_code,municipality_iso_country,departing_from,arrival_time_local,terminal,status,aircraft_model,flight_number,datetime,temperature,wind,prob_perc,rain_qty,snow
0,Berlin Brandenburg Airport,DE,Berlin,EDDB,"Berlin,DE",Duesseldorf,2022-04-18 07:35:00+02:00,1,Unknown,Airbus A319,EW 9048,2022-04-17 15:00:00,15.34,3.19,0.0,0.0,0
1,Berlin Brandenburg Airport,DE,Berlin,EDDB,"Berlin,DE",Duesseldorf,2022-04-18 07:35:00+02:00,1,Unknown,Airbus A319,EW 9048,2022-04-17 18:00:00,13.81,3.34,0.0,0.0,0
2,Berlin Brandenburg Airport,DE,Berlin,EDDB,"Berlin,DE",Duesseldorf,2022-04-18 07:35:00+02:00,1,Unknown,Airbus A319,EW 9048,2022-04-17 21:00:00,9.89,3.18,0.0,0.0,0
3,Berlin Brandenburg Airport,DE,Berlin,EDDB,"Berlin,DE",Duesseldorf,2022-04-18 07:35:00+02:00,1,Unknown,Airbus A319,EW 9048,2022-04-18 00:00:00,5.52,2.31,0.0,0.0,0
4,Berlin Brandenburg Airport,DE,Berlin,EDDB,"Berlin,DE",Duesseldorf,2022-04-18 07:35:00+02:00,1,Unknown,Airbus A319,EW 9048,2022-04-18 03:00:00,4.34,2.07,0.0,0.0,0


## Update data into MySQL database

### `sqlalchemy`

#### Establish the connection

In [14]:
#%pip install sqlalchemy
#%pip install PyMySQL

In [15]:
import pandas as pd
import sqlalchemy

In [28]:
schema="gans"
host="127.0.0.1"
user="root"
# change the password to the root password of your MySQLWorkbench
# (you set this up when you set up MySQL)
password="e^J94QDBUpLjLd1z^2X4XJ"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

#### Update the tables

In [40]:
(
airports_cities
    .dropna()
    .drop_duplicates()
    .to_sql('airports_cities', if_exists='append', con=con, index=False)
)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry 'London,GB' for key 'airports_cities.PRIMARY'")
[SQL: INSERT INTO airports_cities (name, iso_country, municipality, icao_code, municipality_iso_country) VALUES (%(name)s, %(iso_country)s, %(municipality)s, %(icao_code)s, %(municipality_iso_country)s)]
[parameters: ({'name': 'Honiara International Airport', 'iso_country': 'SB', 'municipality': 'Honiara', 'icao_code': 'AGGH', 'municipality_iso_country': 'Honiara,SB'}, {'name': 'Port Moresby Jacksons International Airport', 'iso_country': 'PG', 'municipality': 'Port Moresby', 'icao_code': 'AYPY', 'municipality_iso_country': 'Port Moresby,PG'}, {'name': 'Keflavik International Airport', 'iso_country': 'IS', 'municipality': 'Reykjavík', 'icao_code': 'BIKF', 'municipality_iso_country': 'Reykjavík,IS'}, {'name': 'Priština Adem Jashari International Airport', 'iso_country': 'XK', 'municipality': 'Prishtina', 'icao_code': 'BKPR', 'municipality_iso_country': 'Prishtina,XK'}, {'

In [30]:
(
weather_data
    .assign(datetime = lambda x: pd.to_datetime(x['datetime']))
    .to_sql('weather', if_exists='append', con=con, index=False)
)

In [38]:
import numpy as np
(
arrivals
    .replace({np.nan},'unknown')
    .assign(arrival_time_local = lambda x: pd.to_datetime(x['arrival_time_local']),
           municipality_iso_country = 'Berlin,DE')
    .to_sql('arrivals', if_exists='append', con=con, index=False)
)

In [32]:
weather_data.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-04-17 15:00:00,15.34,3.19,0.0,0.0,0,"Berlin,DE"
1,2022-04-17 18:00:00,13.81,3.34,0.0,0.0,0,"Berlin,DE"
2,2022-04-17 21:00:00,9.89,3.18,0.0,0.0,0,"Berlin,DE"
3,2022-04-18 00:00:00,5.52,2.31,0.0,0.0,0,"Berlin,DE"
4,2022-04-18 03:00:00,4.34,2.07,0.0,0.0,0,"Berlin,DE"


In [34]:
arrivals.head()

,departing_from,arrival_time_local,terminal,status,aircraft_model,flight_number,icao_code
0,Duesseldorf,2022-04-18 07:35:00+02:00,1,Unknown,Airbus A319,EW 9048,EDDB
1,Munich,2022-04-18 07:35:00+02:00,1,Unknown,Airbus A319,LH 1926,EDDB
2,Salzburg,2022-04-18 07:40:00+02:00,1,Unknown,Airbus A319,EW 4342,EDDB
3,Budapest,2022-04-18 07:45:00+02:00,0,Unknown,Airbus A320,W6 2315,EDDB
4,Newark,2022-04-18 07:50:00+02:00,1,Unknown,Boeing 767-300,UA 962,EDDB


In [26]:
airports_cities.sample(100)

,name,iso_country,municipality,icao_code,municipality_iso_country
36288,McGhee Tyson Airport,US,Knoxville,KTYS,"Knoxville,US"
23096,Kenneth Kaunda International Airport,ZM,Lusaka,FLKK,"Lusaka,ZM"
64351,Rajiv Gandhi International Airport,IN,Hyderabad,VOHS,"Hyderabad,IN"
57271,Khabarovsk Novy Airport,RU,Khabarovsk,UHHH,"Khabarovsk,RU"
25636,Roberts International Airport,LR,Monrovia,GLRB,"Monrovia,LR"
...,...,...,...,...,...
39472,Norman Manley International Airport,JM,Kingston,MKJP,"Kingston,JM"
47417,Gimpo International Airport,KR,Seoul,RKSS,"Seoul,KR"
34223,Louis Armstrong New Orleans International Airport,US,New Orleans,KMSY,"New Orleans,US"
68177,Shanghai Hongqiao International Airport,CN,Shanghai (Minhang),ZSSS,"Shanghai (Minhang),CN"
